# Scientific Machine Learning: PINN For Audio Processing





**ASSA 2025 Autumn school**

Mirco PEZZOLI - Politecnico di Milano, Milan, Italy

> Note: this notebook is based on my [Forum Acusticum 2025 summer school](https://github.com/m-pexx/fa25-summer-school) and the [EUSIPCO 2025 tutorial](https://github.com/Chutlhu/EUSIPCO25_PIML_tutorial) on PIML for audio processing
[Credits include Doc. [Diego Di Carlo](https://diegodicarlo.com/index.html) and Prof. [Shoichi Koyama](https://www.sh01.org/)]

## Assignment
- Implement a **Physics-Informed Neural Network (PINN)** for inverse problem solving. In particular, for interpolating RIRs using a PI-SIREN as proposed in "Implicit neural representation with physics-informed neural networks for the reconstruction of the early part of room impulse responses" by Pezzoli et al. presented at Forum Acusticum 2023. Read the paper [here](https://www.researchgate.net/profile/Mirco-Pezzoli/publication/377549887_Implicit_Neural_Representation_With_Physics-Informed_Neural_Networks_For_The_Reconstruction_Of_The_Early_Part_Of_Room_Impulse_Responses/links/665995c6479366623a3382e3/Implicit-Neural-Representation-With-Physics-Informed-Neural-Networks-For-The-Reconstruction-Of-The-Early-Part-Of-Room-Impulse-Responses.pdf).

- The model is based on a Physics-informed SIREN, which is a MLP with sinusoidal activations. Read about SIREN on the original paper [here](https://arxiv.org/abs/2006.09661).




# Let's start!

In [ ]:

# We start importing some libraries
import time
import random
import math
import sys
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import scipy.io.wavfile as wavfile

sys.path.append('.')

In [ ]:
# If GPU (if running in Colab: Runtime -> Change runtime type -> GPU (recommended))
#!nvidia-smi  # Uncomment to see GPU

In [ ]:
# Run on the available device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Current device:', device)

# Set the seed number
def reset_seeds(seed=123):
    """Reset all random seeds to e nsure reproducibility"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

SEED =
reset_seeds(SEED)

All set with the necesarry settings. Let start! 🚀


## InvPINN Model Class

Before focuing on the RIR reconstrcution problem, let's setup the models that we are going to use in the training. In particular, we adapt the class `PINN_Wave1D` class for inverse problems

In practice we **extend the class** to include data loss, as sampling and derivatives functions remain the same.

First, import the PINN_Wave1D class from the previous notebook

In [ ]:
# Copy-paste the class PINN_Wave1D from previous notebook



Secondly, we import the PINN_Wave1D_inverse class in order to have a suitable MLP that deals with data loss

> Hint: have a look at this NN class in the notebook on the PINN introduction



In [ ]:
class PINN_Wave1D_inverse(PINN_Wave1D):
    def __init__(self,
        model_arch,
        wave_speed=1.0, wave_mode=1, domain_length=1.0,  # <-- used for viz. purpose
        samples_sizes = {'N_pde': 2048,},
        loss_weights = {'pde' : 1., # <-- need to add a weight for the dataloss!
        device='cpu',
        ):
        super().__init__(model_arch, wave_speed, wave_mode, domain_length, samples_sizes, loss_weights, device)
        self.w_pde = loss_weights['pde']
        self.w_data = # <-- need to add a loss on the data!

    def train(self,
              tz_data,  # <-- need to pass the data in the training function
              n_iters=1500, lr=1e-3, log_every=100, plot_live=False):
        """Train the PINN model"""
        self.model.train()
        opt = # <-- set the optimizer!
        hist = []

        # Evaluation points for live plotting
        if plot_live:
            z_eval = torch.linspace(0, self.L, 400)[:, None]
            t_eval = 0.35
            tz_eval = torch.cat([torch.full_like(z_eval, t_eval), z_eval], dim=1).to(self.device)

        for step in range(n_iters):

            opt.zero_grad(set_to_none=True)

            # (1) PDE loss
            tz_c = # <-- retrieve the collocation points (i.e., domain over which compute the PDE loss)
            u_c =  # <-- get the estimte of from the NN
            ut_c, uz_c, utt_c, uzz_c = # <-- get the derivates!
            r =  # <-- implement PDE residual
            L_pde = torch.mean(r**2)

            # (4) Data / Supervised losses
            u_d = # <-- Get the model prediction!
            L_data = torch.mean((u_d - u_data)**2)

            # (6) total loss & step
            loss = # <-- combine the two losses weighted
            loss.backward()
            opt.step()

            # some loggings
            if step % log_every == 0:
                parts = {
                    'pde': float(L_pde.detach().cpu()),
                    'data': float(L_data.detach().cpu()),
                }
                hist.append((step, float(loss.detach().cpu()), parts))
                print(f"[Adam {step:04d}] total={hist[-1][1]:.3e} parts={parts}")

                if plot_live:
                    self._plot_live_training(hist, tz_eval, z_eval, t_eval, step=step, total_steps=n_iters)

        if plot_live:
            self._plot_live_training(hist, tz_eval, z_eval, t_eval, step=step, total_steps=n_iters)

        u_pred = self.model(tz_eval)
        u_ref = self.analytical_solution(t_eval, z_eval.to(self.device))
        mse = torch.mean((u_pred - u_ref)**2).mean()
        print(f'Training ended. Final MSE {mse.item()}')

        return hist


# SIREN model

We want to implement a PI-SIREN which is a MLP with sinusoidal activation and train it to estimate RIRs in a uniform linear array.

SIREN has the structure of MLP defined by several layer $\phi$ as

$g(\boldsymbol{x} ; \boldsymbol{\theta})=\left(\phi_L \circ \phi_{L-1} \circ \cdots \circ \phi_1\right)(\boldsymbol{x})$

where $x$ is the input of the MLP and $\theta$ are the learnable parameters of the model.


Each $i$th layer is characterized by sinusoidal activation leading to

$\phi_i\left(\boldsymbol{x}_i\right)=\sin \left(\omega_0 \boldsymbol{x}_i^T \boldsymbol{\theta}_i+\boldsymbol{b}_i\right)$

where $\omega_0$ is a hyperparameter that controls the freuquency of the sinusoidal function. For more details about the impact of $\omega_0$ read the [paper](https://arxiv.org/abs/2006.09661) by Sitzman.

**Why** (vs tanh/ReLU MLPs)
- Battles **spectral bias** → captures high-frequency/detail (e.g., Helmholtz high-k, sharp boundaries)
- Stable gradients over space/time for INRs & PINNs
- Originally proposed for image super-resolution and also PINNs

**When to use**
- Coordinate-based fields (signal/image/audio)
- PDEs with **oscillatory** solutions, multi-scale detail.

**Tips for PINNs**
- Normalize inputs coordinates to $[-1,1]$
- $\omega_0$ should match the "bandwidth" of the signal, i.e., the higher the frequency content, the bigger the $\omega_0$

In [ ]:
# Implement the SIREN model
# SIREN layer and network (great for oscillatory signals)
class Sine(nn.Module):
    def __init__(self, w0=30.0):
        super().__init__()
        self.w0 = w0
    def forward(self, x):
        return # <-- implement the Sinusoidal layer

def siren_init(m, w0): # SIREN requires particular initialization
    if isinstance(m, nn.Linear):
        in_f = m.weight.shape[1]
        bound = 1./in_f if hasattr(m, 'is_first') else math.sqrt(6/in_f)/w0
        with torch.no_grad():
            m.weight.uniform_(-bound, bound);
            if m.bias is not None: m.bias.uniform_(-bound, bound)

class SirenNet(nn.Module):
    def __init__(self, in_dim=2, hidden=64, depth=4, w0 # <-- Set initial omega0
                 , w0_hidden=1.0):
        super().__init__()
        layers=[]
        for i in range(depth):
            lin = nn.Linear(in_dim if i==0 else hidden, hidden)
            if i==0:
                lin.is_first=True
                layers += [lin, # <-- first layer use w0
            else:
                layers += [lin, # <-- others use w0_hidden
        layers += # <-- last layer is linear!

        self.net = nn.Sequential(*layers)
        self.apply(lambda m: siren_init(m, w0_hidden))

    def forward(self, tx): return self.net(tx)

## Room Impulse Response (RIR) reconstruction

Let's apply PI-SIREN for **sound field reconstruction**, in particular spatial upsampling of measurement Room Impulse Responses (RIRs).


### How to proceed
1. Train the PINN without removing any channel from the array and using only the data loss. This data fit will show if architecture works.
2. Impose the random mask to remove channels from the ULA and train the network.
3. Add the PDE loss to the training, what can you observe? You probably have to play with weighting factors $\lambda$



### Get the data

In [ ]:
import scipy.io.wavfile as wavfile
!wget https://github.com/m-pexx/fa25-summer-school/raw/refs/heads/main/data/simulated_rir2.wav

fs, data = wavfile.read("./simulated_rir2.wav")
print(data.shape)

# normalized
data = # <-- peak normalize the data
data =  # <--- Get the first 200 time samples to keep small size
n_samples, n_mics = data.shape

# let's create a random mask selecting 70% of the mics
mask_np = np.ones(data.shape)
N_mics_to_remove = # <-- Set the number of mics to remove
print(N_mics_to_remove)
idx_remove = np.sort(np.random.choice(np.arange(data.shape[1]), N_mics_to_remove, replace=False))
idx_to_keep = np.setdiff1d(np.arange(data.shape[1]), idx_remove)
mask_np[:,idx_remove] *= 0

fig, axarr = plt.subplots(1,2, figsize=(7,3), sharey=True)
axarr[0].imshow(data, aspect='auto', cmap="seismic", vmax=1, vmin=-1)
axarr[0].set_xlabel('Channel index')
axarr[0].set_ylabel('Samples')
axarr[1].imshow(mask_np * data , aspect='auto', cmap="seismic", vmax=1, vmin=-1)
plt.show()

### Adapt the PINN

In [ ]:
# @title
class PINN_RIRs(PINN_Wave1D_inverse):
    def __init__(self,
        model_arch,
        samples_sizes = {'N_pde': 2048,},
        loss_weights = {'pde' : 1., 'data' : 1.,},
        device='cpu',
    ):
        super().__init__(model_arch, 1, 1, 1, samples_sizes, loss_weights, device)
        self.w_pde = loss_weights['pde']
        self.w_data = loss_weights['data']
        self.c = 343.                        # <--- change the speed of sound

    def sample_collocation(self, Nc):
        """Sample collocation points in domain [0,1] x [0,L]"""
        t = torch.rand(Nc, 1) * 2 - 1      # <--- change the input domain in [-1,1]
        z = torch.rand(Nc, 1) * 2 - 1      # <--- change the input domain in [-1,1]
        return torch.cat([t, z], dim=1).to(self.device)

    def train(self, tz_data, u_data, tz_ref, u_ref, n_iters=1500, lr=1e-3, log_every=100, plot_live=False):
        """Train the PINN model"""
        self.model.train()
        opt = torch.optim.Adam(self.model.parameters(), lr=lr)
        hist = []

        # Evaluation points for live plotting
        for step in range(n_iters):

            opt.zero_grad()

            # (1) PDE loss
            tz_c = # <-- retrieve the collocation points (i.e., domain over which compute the PDE loss)
            u_c =  # <-- get the estimte of from the NN
            ut_c, uz_c, utt_c, uzz_c = # <-- get the derivates!
            r =  # <-- implement PDE residual
            L_pde = torch.mean(r**2)

            # (4) Data / Supervised losses
            u_d = # <-- Get the model prediction!
            L_data = torch.mean((u_d - u_data)**2)

            # (6) total loss & step
            loss = # <-- combine the two losses weighted
            loss.backward()
            opt.step()

            if step % log_every == 0:
                parts = {
                    'pde': float(L_pde.detach().cpu()),
                    'data': float(L_data.detach().cpu()),
                }
                hist.append((step, float(loss.detach().cpu()), parts))
                print(f"[Adam {step:04d}] total={hist[-1][1]:.3e} parts={parts}")

                with torch.no_grad():
                    u_pred = self.model(tz_ref).cpu().numpy().reshape(*u_ref.shape)

                if plot_live:
                    self._plot_live_training(hist, u_pred, u_ref.detach().cpu().numpy(), step, n_iters)
                    print("Loss: %0.6f MSE: %0.6f PDE %0.6f" %(loss.item(), (self.w_pde*L_pde).item(), (self.w_data*L_data).item()))

        if plot_live:
            self._plot_live_training(hist, u_pred, u_ref.detach().cpu().numpy(), step, n_iters)

        with torch.no_grad():
            u_pred = self.model(tz_ref).reshape(*u_ref.shape)
        mse = torch.mean((u_pred - u_ref)**2)
        print("Final Loss: %0.6f MSE: %0.6f PDE %0.6f" %(loss.item(), (self.w_data*L_data).item(), (self.w_pde*L_pde).item()))
        print(f"Model trained. Final full field reconstruction MSE: {mse.item():0.6f}")
        return hist

    def _plot_live_training(self, hist, u_pred, u_ref, step, total_steps, title_suffix=""):

        try:
            from IPython.display import clear_output
            clear_output(wait=True)
        except ImportError:
            pass  # Skip clearing output if IPython not available

        steps, totals, comps = zip(*hist)
        pde = [c['pde'] for c in comps]
        data = [c['data'] for c in comps] if 'data' in comps[0] else None

        fig, axarr = plt.subplots(1,4, figsize=(14,5))
        axarr[0].imshow(u_ref, aspect='auto', cmap="seismic", vmin=-1.1, vmax=1.1)
        axarr[0].set_xlabel('Channel index')
        axarr[0].set_ylabel('Samples')
        axarr[0].set_title('Reference')

        axarr[1].imshow(u_pred , aspect='auto', cmap="seismic", vmin=-1.1, vmax=1.1)
        axarr[1].set_xlabel('Channel index')
        axarr[1].set_ylabel('Samples')
        axarr[1].set_title('Prediction')
        axarr[2].plot(steps, pde, label='PDE')
        if data is not None:
            axarr[2].plot(steps, data, label='Data')
        axarr[2].set_xlim([0, total_steps])
        axarr[2].set_title("Loss Components")
        axarr[2].set_xlabel("Steps")
        axarr[2].legend()
        axarr[2].grid(True)

        axarr[3].plot(steps, self.w_pde * np.array(pde), label='PDE')
        if data is not None:
            axarr[3].plot(steps, self.w_data * np.array(data), label='Data')
        axarr[3].set_xlim([0, total_steps])
        axarr[3].set_title("Weigthed Loss Components")
        axarr[3].set_xlabel("Steps")
        axarr[3].legend()
        axarr[3].grid(True)

        plt.suptitle(f'Training loop {step}/{total_steps} iters' + title_suffix)
        plt.tight_layout()
        plt.show()
        return

### And train it

In [ ]:
z_coords = # <-- space samples
t_coords = # <-- time samples

tz_grid = np.stack(np.meshgrid(t_coords, z_coords, indexing='ij'), -1)
print(tz_grid.shape, data.shape)

# define points for qualitative evaluation
tz_true = torch.tensor(tz_grid, dtype=torch.float32, device=device)
u_true = torch.tensor(data, dtype=torch.float32, device=device)

# define observed points
tz_obs = torch.tensor(tz_grid[:,idx_to_keep].reshape(-1,2), dtype=torch.float32, device=device)
u_obs = torch.tensor(data[:,idx_to_keep].reshape(-1,1), dtype=torch.float32, device=device)


reset_seeds(SEED)
model_arch = # <-- Define the SIREN networks! Suggested parameters # hidden 128, 5 layers in total, initial omega 30 and hidden omega as half of w0
model_inv = PINN_RIRs(
    model_arch,
    loss_weights= # <-- train withouth the PDE loss for testing
    samples_sizes = {'N_pde': 2048},
    device=device
)

start = time.time()
### TRAINING WITHOUT GPU is quite slow, is better to use GPU computing
_ = # <-- train for 600 epochs and learning rate 0.001
print(f"PINN training: elapsed time: {time.time()-start:.2f} s")

With loss PDE's weigths = 0, we overfit!

In [ ]:
z_coords = # <-- space samples
t_coords = # <-- time samples # <-- Play with the input range e.g., [-10,10] it's equaivalent to change w0 for one dimension

tz_grid = np.stack(np.meshgrid(t_coords, z_coords, indexing='ij'), -1)
print(tz_grid.shape, data.shape)

# define points for qualitative evaluation
# We train the model on GPU, thus we need to load the data on the device memory
tz_true = torch.tensor(tz_grid, dtype=torch.float32, device=device)
u_true = torch.tensor(data, dtype=torch.float32, device=device)

# define observed points
tz_obs = torch.tensor(tz_grid[:,idx_to_keep].reshape(-1,2), dtype=torch.float32, device=device)
u_obs = torch.tensor(data[:,idx_to_keep].reshape(-1,1), dtype=torch.float32, device=device)


reset_seeds(SEED)
# <-- Play with w0, w0_hidden, number of hitten layers and depth!
model_arch = # <-- Define the SIREN networks!
model_inv = PINN_RIRs(
    model_arch,
    loss_weights= # <-- find good balance for PDE loss (suggested weight .5e-15)
    samples_sizes = {'N_pde': 2048},
    device=device
)

start = time.time()
### TRAINING WITHOUT GPU is quite slow
_ = # <-- train for 600 epochs and learning rate 0.001: try to play with the parameters
print(f"PINN training: elapsed time: {time.time()-start:.2f} s")

🧐 Notice:
- **Lower reconstruction MSE**
- Effective **interpolation** of missing sensor data
- **Lower PDE** violation loss (not 0, though because it's penalized model)